# pypbl
## Preference based learning with python

## What is preference based learning?

If we know a set of preferences based on pairwise comparison 

* **item a** ≻ **item b**
* **item b** ≻ **item d**

What can we say about **item c**?


## Use cases

### Recommender system
We wish to suggest new items that are of interest based on an individual's personal preferences.

For example, recommending a car make and model based on subjective comparison of car attributes.

### Decision support
A decision maker has to make a decision based on multiple objectives. Preference based learning can be used to understand the user’s preferences for trading off different objectives or KPIs.

For example, designing air traffic avoidance algorithms based on subject matter expertise.

### Parameter inference
We wish to infer parameters of a model with only binary information.

For example, rating X-box player skill level based on histoic wins and losses between players.

## Cars example

In [1]:
import pandas as pd
from pypbl.priors import Normal, Exponential
from pypbl.elicitation import BayesPreference

In [2]:
data = pd.read_csv('../examples/data/mtcars.csv')
data.set_index('model', inplace=True)
data.head(10)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
model,,,,,,,,,,,
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [3]:
p = BayesPreference(data=data)
p.set_priors([
    Exponential(1),  # MPG - high miles per gallon is preferred
    Normal(),        # number of cylinders
    Normal(),        # displacement
    Exponential(2),  # horsepower - high horsepower is preferred
    Normal(),        # real axle ratio
    Normal(),        # weight
    Exponential(-3), # quarter mile time - high acceleration is preferred
    Normal(),        # engine type
    Normal(),        # transmission type
    Normal(),        # number of gears
    Normal()         # number of carburetors
])

In [4]:
# add some preferences and infer the weights for each parameters
p.add_strict_preference('Pontiac Firebird', 'Fiat 128')
p.add_strict_preference('Mazda RX4', 'Mazda RX4 Wag')
p.add_indifferent_preference('Merc 280', 'Merc 280C')

In [5]:
p.infer_weights(method='mean')

print('inferred weights \n')
for a, b in zip(data.columns.values.tolist(), p.weights.tolist()):
    print('{}: {}'.format(a, b))

/Users/jim.parr/repos/pypbl/binder/venv/lib/python3.8/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in double_scalars
  lnpdiff = f + nlp - state.log_prob[j]


inferred weights 

mpg: 0.33209059190606105
cyl: 0.25923213575607135
disp: 0.48261880291726117
hp: 0.5133632088932872
drat: -0.02267199712683362
wt: -0.45621913917700246
qsec: -1.07077931064681
vs: -0.08519745913839491
am: -0.33196719193333807
gear: -0.23029215990580393
carb: 0.22260136485081916


In [6]:
# rank all the items and highlight the top five
print('\ntop 5 cars')
p.rank().head(5)


top 5 cars


,utility
Duster 360,0.692171
Camaro Z28,0.683887
Maserati Bora,0.513858
Pontiac Firebird,0.434025
Hornet Sportabout,0.429142


In [7]:
# suggest a new set of comparisons
p.suggest(method='max_variance')

('Duster 360', 'Ford Pantera L')